In [ ]:
import numpy as np
import csv

from astropy.timeseries import BoxLeastSquares
from scipy.signal import savgol_filter
from astroquery.mast import Observations
import scipy.fftpack
from scipy import signal
from scipy.signal import savgol_filter
from scipy.stats import binned_statistic
from dave.tessPipeline.tessmastio import TessAstroqueryArchive
import dave.fileio.tpf as tpf
import dave.tessPipeline.tessfunc as tessfunc
from dave.tessPipeline import vet_tess_
import dave.tessPipeline.tessPipeline as tessPipeline
import dave.pipeline.clipboard as clipboard
from dave.tessPipeline.pertransitcentroids import measurePerTransitCentroids
from dave.tessPipeline.pertransitcentroids import getIngressEgressCadences
from dave.tessPipeline.pertransitcentroids import generateDiffImg
import dave.diffimg.tessprf as prf
from dave.diffimg.tesscent import getBoundingBoxForImage
from dave.diffimg.tesscent import fitPrfCentroidForImage
from dave.diffimg.tesscent import plotCentroidFitDiagnostic
import dave.misc.covar as covar
import pandas as pd
from astroquery.mast import Catalogs
import eleanor
import lightkurve as lk
from mpl_toolkits.axes_grid1 import make_axes_locatable
import pickle
#from PyAstronomy import pyasl
from scipy.signal import savgol_filter
from astropy.coordinates import SkyCoord
from astroquery.mast import Tesscut
from tess_stars2px import tess_stars2px_function_entry as tess_stars2px
from astropy.io import fits
from astropy.wcs import WCS
from statistics import mode
from astropy.coordinates import SkyCoord
from astropy import coordinates as coord, units as u
import os
import glob
from astropy.modeling import models, fitting
from scipy.optimize import minimize, rosen, rosen_der
from astropy.modeling.models import custom_model
from scipy.optimize import curve_fit
#import contaminante as ct

import helper_codes
import helper_codes_EBs
import importlib

import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
plt.rcParams.update({'font.size': 20})

np.set_printoptions(precision=4)

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

In [ ]:
#fn = '/Users/vkostov/.eleanor/mastDownload/TESS/tess2019199201929-s0014-s0026-0000000235678745/tess2019199201929-s0014-s0026-0000000235678745-00353_dvt.fits'
#data = fits.getdata(fn)
#time, flux, model = data['TIME'], data['LC_DETREND'], data['MODEL_INIT']

#plt.figure(figsize=(20,5))
#plt.plot(time, flux, 'k.', time, model, 'r')
#plt.xlim(27*nn + min(time), 27*(nn+1) + min(time))
#plt.xlim(1870+30, 1900+30)
#plt.xlim(1955., 2035)
#plt.show()

#idx, = np.where((time >=1955.) & (time <=2035.))
#print(time[idx].shape, model[idx].shape)
#np.savetxt('/Users/vkostov/Desktop/Ideas_etc/DAVE/dave/vetting/tmp_235678745.csv', np.c_[time[idx], flux[idx], model[idx]], delimiter = ',')
#./modshift tmp_235678745.csv TIC_235678745 TOI-2095.01 17.664629 1692.815013 1

In [ ]:
#eleanor.__version__# AS OF OCT 28 2020 ELEANOR 1.0.5 WORKS FOR CYCLE 1 & 2!!!!!!
#eleanor.__path__

In [ ]:
# CALCULATE ANGULAR SEPARATION FROM RA AND DEC
ra1 = 285.3693205677977
dec1 = 24.92619939627369
ra2 = 285.3690613948402
dec2 = 24.92609405776401

ang_sep = helper_codes.calculate_angular_separation(ra1, dec1, ra2, dec2)
#print(ang_sep)

In [ ]:
k = 0.01720209895 # Gaussian Grav Constant
gm = k**2 # units of AU, DAYS and solar masses
rsun_to_AU = 0.00465047
year = np.sqrt((4.*np.pi**2)/gm)
solar_radius_to_au = 0.00465047 # AU

#print(year)

In [ ]:
# Stellar Mass-Radius relationship
def radius(m): # from https://academic.oup.com/mnras/article/467/2/2160/2929274
    return 10**(1.4296*np.log10(m)+1.5792*(np.log10(m)**2)+2.8198*(np.log10(m)**3)+3.0405*(np.log10(m)**4)+1.2841*(np.log10(m)**5))

In [ ]:
#aa = 0.11
#mm = 10.
#per_ = np.sqrt((4.*np.pi**2)*(aa**3)/gm/mm)
#per_#/year
#per_ = 28.# days
#aa = ((4.*np.pi**2)**(1./3))*((per_/365.)**(2./3))/(2*gm)**(1./3)
#aa

In [ ]:
#alpha = 0.8
#d = 150*1e-3 # in Kpc
#a = d*alpha/206265
#a*206265, 'AU'

In [ ]:
#path_ = '/home/neptune/Downloads/'
#fn_ = path_ + 'Quad Catalog #2 - sorted_by_pass.csv'
#aa = pd.read_csv(fn_, header = 0, delimiter=",",engine='python',na_filter=True).values
#TIC_ = aa[:,0].astype(int)

#fn_zasche = path_ + 'Quad Catalog #2 - Zasche_2022.csv'
#aa_zasche = pd.read_csv(fn_zasche, header = 0, delimiter=",",engine='python',na_filter=True).values
#TIC_zasche = aa_zasche[:,0].astype(int)

#same_TICs = np.intersect1d(TIC_zasche, TIC_)
#same_TICs

In [ ]:
#eleanor.Update(76)
#eleanor.__file__
#eleanor.__version__

In [ ]:
TIC = 274996674
print('https://exofop.ipac.caltech.edu/tess/nearbytarget.php?id=' + str(TIC))
catalog_ = helper_codes.get_catalog_info(int(TIC)); catalog_

In [ ]:
importlib.reload(eleanor)
import eleanor

coord = SkyCoord(catalog_.ra.values[0], catalog_.dec.values[0], unit = "deg")
result = tess_stars2px(TIC, coord.ra.value, coord.dec.value)
sector_all_ = result[3]; print('TIC',TIC,'observed in Sectors',sector_all_,'. Removing S65 (eleanor crashes) and S76+ (not ready yet)...')
bad_sectors = np.logical_or(sector_all_ == 65, sector_all_ > 77)
sector_all_ = sector_all_[~bad_sectors].tolist()

path_ = '/Users/vkostov/Desktop/Ideas_etc/TESS/EBs/Quad_catalog/data/qcfits2/'
arr_ = np.sort(glob.glob(path_ + '*' + str(TIC) + '*.fits'))

try:
# if 1==1:
    # star_TC = eleanor.multi_sectors(tic=int(TIC), tc = True, sectors = sector_all_)#sector_all_[0:-1])#list(sectorTable['sector'].value))
    #star_TC = eleanor.multi_sectors(tic=int(TIC), tc = True, sectors='all')
    star_TC = eleanor.multi_sectors(tic=int(TIC), tc = True, tesscut_size = 31, sectors=[10])

    if len(arr_) <  len(star_TC):
        for s in star_TC:
            #print(s.sector)
            data = eleanor.TargetData(s, do_psf = False, do_pca = False)#, aperture_mode = 'small')#, height = 11, width = 11)
            data.save(directory = path_)
        arr_ = np.sort(glob.glob(path_ + '*' + str(TIC) + '*.fits'))
    print()
    print('eleanor done')
except:
# else:
    print('eleanor fails for sector zzzz')

In [ ]:
eleanor_visualization = False#True#
eleanor_show_aperture = False#True#
eleanor_show_gaia = False#True#
elearno_pixel_by_pixel = True#False#
if eleanor_visualization:
    star_TC_tmp = star_TC[0]#eleanor.multi_sectors(tic=int(TIC), tc = True, sectors=[38])
    data_tmp = eleanor.TargetData(star_TC_tmp, do_psf = False, do_pca = True)
    vis = eleanor.Visualize(data_tmp)
    if eleanor_show_aperture:
        vis.aperture_contour()
    if eleanor_show_gaia:
        fig_gaia = vis.plot_gaia_overlay(magnitude_limit=15)
    if elearno_pixel_by_pixel:
        fig = vis.pixel_by_pixel(data_type="corrected", ylim=[0.99,1.01], colrange=[3,10],rowrange=[3,10])#,xlim =[2980,2990])
        # plt.axvline(1692., color = 'b', ls = '--')

In [ ]:
which_sector = input('Which Sectors (default = All Available): ')
if len(which_sector) == 0:
    which_sector = np.linspace(0,len(arr_)-1,len(arr_)).astype(int)
else:
    which_sector = [int(i) for i in which_sector.split(',')]

do_plot_ = 'y'

importlib.reload(helper_codes)
tt, rr_, cc_, pca_, qq_, cube_, TIC, sector_, camera_, ccd_, tpf_x, tpf_y, hdr_ = \
helper_codes.get_data_(arr_, which_sector, do_plot_)#, how_many_sectors, skip_sectors, do_plot_)

bad_times_ = helper_codes.bad_times_()

In [ ]:
get_ATLAS = 'n'

if get_ATLAS == 'y':
    filename_ = '/Users/vkostov/Desktop/Ideas_etc/TESS/TIC_391461666_pCBP/TIC_' + str(TIC) + '_ATLAS.txt'
    importlib.reload(helper_codes)
    
    period, t0 = 8.000103, 1792.6335
    dict_ = helper_codes.get_ATLAS_data(filename_, period, t0)
    time_ATLAS_, flux_ATLAS_, phase_ATLAS_sorted, flux_ATLAS_sorted = \
    dict_['time_ATLAS'], dict_['flux_ATLAS'], dict_['phase_ATLAS_sorted'], dict_['flux_ATLAS_sorted']

    do_plot_ = 'n'
    if do_plot_ == 'y':
        fig = plt.figure(figsize=(20,5))
        plt.plot(phase_ATLAS_sorted, flux_ATLAS_sorted, 'r.', label = 'ATLAS')
        plt.legend()
        plt.xlim(-0.4, 0.6)
        plt.ylim(0.9, 1.05)
        plt.xlabel('Phase Binary A')
        plt.ylabel('Normalized Flux')

In [ ]:
get_ZTF = 'n'
if get_ZTF == 'y':
    importlib.reload(helper_codes)
    do_plot_ = 'n'
    dict_ = helper_codes.get_ZTF_data(TIC, do_plot_)

    tt_ZTF, ff_ZTF = dict_['tt_ZTF'], dict_['ff_ZTF']

    fig = plt.figure(figsize=(20,5))
    phase_ztf = tt_ZTF/31.847 % 1  
    plt.plot(tt_ZTF, ff_ZTF, 'ko')
    plt.title('TIC 351404069, ZTF')
    plt.xlabel('BJD - 2,457,000')

    #np.savetxt('TIC_' + str(TIC)+'_ZTF.csv', np.c_[tt_ZTF, ff_ZTF], delimiter = ',')

In [ ]:
get_Kepler_data = False
if get_Kepler_data:
    kic_id = 'KIC 10508190'
    try:
        print('Try original Kepler')
        search_result = lk.search_lightcurve(kic_id, author='Kepler')
    except:
        print('Try Kepler Bonus')
        fname = lk.search_lightcurve(kic_id, cadence="long", mission="Kepler", author="KBONUS-BKG").download().filename
        hdul = fits.open(fname)
        hdul.info()
        hdul["PRIMARY"].header
        hdul["LIGHTCURVE_STITCHED"].data

In [ ]:
fig = plt.figure(figsize=(20,11))
plt.rcParams.update({'font.size': 26})  

color_ = ['r', 'g', 'b']

for ii in range(len(which_sector)):
    importlib.reload(helper_codes)
    do_plot_ = 'n'
    tt, rr_, cc_, pca_, qq_, cube_, TIC, sector_, camera_, ccd_, tpf_x, tpf_y, hdr_ = \
    helper_codes.get_data_(arr_, [which_sector[ii]], do_plot_)

    tt_ref_ = tt[0]
    custom_bad_times = np.ones(len(tt), dtype=bool)
    for bb in range(len(bad_times_)):
        custom_bad_times[(tt - 0.*tt_ref_ >= bad_times_[bb,0]) & (tt - 0.*tt_ref_ <= bad_times_[bb,1])] = False

    tt = tt[custom_bad_times]
    cc_ = cc_[custom_bad_times]#1. - ff_[m]
    qq_ = qq_[custom_bad_times]

    plt.subplot(len(which_sector),1,ii+1)
    
    if ii > 10:
        plt.plot(0.*2e3+tt[~qq_.astype(bool)], cc_[~qq_.astype(bool)], 'k')
        plt.xticks([])
    else:
        tt_tmp_, ff_tmp_ = tt[~qq_.astype(bool)], cc_[~qq_.astype(bool)]
        
        zz_ = np.zeros((len(tt_tmp_))).astype(bool)
        plt.plot(0.*2e3+tt_tmp_[~zz_], ff_tmp_[~zz_], 'k')#, label = 'Sector ' + str(sector_))#, color = color_[ii], ls = '-')
        plt.xlim(min(tt_tmp_[~zz_]), max(tt_tmp_[~zz_]))
        if ii == 0:
            plt.title('TIC '  + str(TIC) + '; Sector ' + str(sector_[0]))
        else:
            plt.title('Sector ' + str(sector_[0]))
        #plt.legend()
        plt.ylim(0.9,1.05)

        # plt.axvspan(1818, 1818.1, color = 'g', alpha = 0.3)
        # plt.axvspan(2913.4, 2913.6, color = 'g', alpha = 0.3)
        # plt.axvspan(2929.5, 2929.7, color = 'g', alpha = 0.3)
        # plt.axvspan(3294.4, 3294.6, color = 'g', alpha = 0.3)
        # plt.axvspan(3300., 3300.2, color = 'g', alpha = 0.3)

In [ ]:
if TIC == 286779918:

    t0_bin = 1820.0184
    t0s_cbp = np.asarray([1818.08, 2913.5, 2929.6, 3294.5, 3300.1])    
    tmp_tt, tmp_ff = tt[qq_ == 0.], cc_[qq_ == 0.]
    
    fig = plt.figure(figsize=(20,5))
    
    tmp_tt, tmp_ff = tt[qq_ == 0.], cc_[qq_ == 0.]
    per_ = 370.51
    
    tmp_tt_1, tmp_ff_1 = tmp_tt[tmp_tt < 1850], tmp_ff[tmp_tt < 1850]
    pp_tmp_1 = (tmp_tt_1 - 1.*t0s_cbp[0])/per_ % 1
    pp_tmp_1[pp_tmp_1 > 0.5] = pp_tmp_1[pp_tmp_1 > 0.5] - 1.
    
    tmp_tt_2, tmp_ff_2 = tmp_tt[(tmp_tt > 1850) & (tmp_tt < 2940)], tmp_ff[(tmp_tt > 1850) & (tmp_tt < 2940)]
    pp_tmp_2 = (tmp_tt_2 - 1.*t0s_cbp[0])/per_ % 1
    pp_tmp_2[pp_tmp_2 > 0.5] = pp_tmp_2[pp_tmp_2 > 0.5] - 1.
    
    tmp_tt_3, tmp_ff_3 = tmp_tt[tmp_tt > 2940], tmp_ff[tmp_tt > 2940]
    pp_tmp_3 = (tmp_tt_3 - 1.*t0s_cbp[0])/per_ % 1
    pp_tmp_3[pp_tmp_3 > 0.5] = pp_tmp_3[pp_tmp_3 > 0.5] - 1.
    
    plt.plot(pp_tmp_1[pp_tmp_1.argsort()], tmp_ff_1[pp_tmp_1.argsort()], 'k.-', label = 'S19')
    plt.plot(pp_tmp_2[pp_tmp_2.argsort()], -0.01 + tmp_ff_2[pp_tmp_2.argsort()], 'r.-', label = 'S59')
    plt.plot(pp_tmp_3[pp_tmp_3.argsort()], +0.01 + tmp_ff_3[pp_tmp_3.argsort()], 'b.-', label = 'S73')
    plt.ylim(0.98, 1.03)
    plt.xlim(-0.0075, 0.0075)
    plt.legend()
    plt.title('TIC 286779918, P2 = 370.51 days')

In [ ]:
check_eleanor_diagnostics = 'n'
if check_eleanor_diagnostics == 'y':
    data_TC = eleanor.TargetData(star_TC[0], do_psf = True, do_pca = True)
    
    importlib.reload(helper_codes)
    aa = helper_codes.plot_eleanor_diagnostics(data_TC, catalog_)

In [ ]:
try_tglc = 'n'

if try_tglc == 'y':
    plot_tgls_ = True
    importlib.reload(helper_codes)
    time_tglc, flux_tglc = helper_codes.try_TGLC_(TIC, plot_tgls_)

    if plot_tgls_:
        plt.plot(time_tglc, flux_tglc)
        plt.xlim(min(time_tglc), min(time_tglc) + 30)
        plt.ylim(0.8, 1.05)

In [ ]:
interact_sky_ = input('Interact Sky? (y/n, default = "n"): ')
if interact_sky_:
    which_sector_ = input('Which sector? (default = 0): ')
    if which_sector_:
        which_sector_ = int(which_sector_[0])
    else:
        which_sector_ = 0
    tpf_lk = lk.search_tesscut('TIC ' + str(int(TIC)), sector=star_TC[which_sector_].sector).download_all(cutout_size = (7,7))

In [ ]:
if interact_sky_ == 'y':
    tpf_lk[0].interact()#notebook_url='localhost:8888')

In [ ]:
if interact_sky_ == 'y':
    tpf_lk[0][500:].interact_sky(magnitude_limit = 21)#, notebook_url='localhost:8888')

In [ ]:
custom_plot_interact_sky = 'n'
if custom_plot_interact_sky == 'y':
    importlib.reload(helper_codes)
    remove_duplicate_TIC = 2
    delta_mag = 6
    aa = helper_codes.custom_plot_interact_sky(catalog_, TIC, cube_, remove_duplicate_TIC, delta_mag)

In [ ]:
try_contaminante = 'n'

if try_contaminante == 'y':

    tpfs = lk.search_targetpixelfile('TIC 387288959', sector=16).download()
    period = 2.69633
    t0 = 1739.4627
    duration = 3.3/24

    result = ct.calculate_contamination(tpfs, period, t0, duration, plot=True)

In [ ]:
importlib.reload(helper_codes)
bad_times_ = helper_codes.bad_times_()

which_sector = input('Which Sectors (default = All Available): ')
if len(which_sector) == 0:
    which_sector = np.linspace(0,len(arr_)-1,len(arr_)).astype(int)
else:
    which_sector = [int(i) for i in which_sector.split(',')]

tt, rr_, cc_, pca_, qq_, cube_, TIC, sector_, camera_, ccd_, tpf_x, tpf_y, hdr_ = \
helper_codes.get_data_(arr_, which_sector, do_plot_ = 'n')
tt_, ff_ = tt[~qq_.astype(bool)], cc_[~qq_.astype(bool)]

detrend_ = input('Detrend Data (default = n)? ')    
if len(detrend_) == 0:
    dict_ = helper_codes.run_BLS_test(tt_, ff_, which_sector, TIC, bad_times_, do_plot_, get_disentangled_lc = 'n')
    tt_, tt_ref_, ff_, bls_periods, bls_t0s, bls_durations, bls_depths, transit_mask_ = \
    dict_['tt_'], dict_['tt_ref_'], dict_['ff_'], dict_[
    'bls_periods'], dict_['bls_t0s'], \
    dict_['bls_durations'], dict_['bls_depths'], dict_['transit_mask_']
else:
    x_detrend_, y_detrend_, x_ref, x_bak_, y_bak_, smooth, m = \
    helper_codes.detrend_data_(tt_, ff_, min(tt_), bls_periods, bls_t0s, bls_durations, do_plot_ = 'y')   
    tt_, ff_ = x_detrend_ + 0.*x_ref, y_detrend_# + 1.

    dict_ = helper_codes.run_BLS_test(tt_, ff_, which_sector, TIC, bad_times_, do_plot_, get_disentangled_lc = 'n')

    tt_, tt_ref_, ff_, bls_periods, bls_t0s, bls_durations, bls_depths, transit_mask_ = \
    dict_['tt_'], dict_['tt_ref_'], dict_['ff_'], dict_['bls_periods'], dict_['bls_t0s'], \
    dict_['bls_durations'], dict_['bls_depths'], dict_['transit_mask_']

In [ ]:
# pseudo_sync = helper_codes.calculate_pseudosynchronous_rotation(bls_periods[3], 0.15)
# print(pseudo_sync/2.325, bls_periods)

In [ ]:
remove_all_eclipses = False
if remove_all_eclipses:
    transit_mask_all = np.sum(transit_mask_, axis = 0).astype(bool)
    tt_[transit_mask_all].shape
    pp_ = tt_[transit_mask_all] / 2.3245 % 1
    
    fig = plt.figure(figsize=(20,5))
    plt.plot(pp_, ff_[transit_mask_all], '.')

In [ ]:
calc_ecc_ = False
if calc_ecc_:

    nn_, mm_ = 3,4
    dur_prim, dur_sec, T0prim, T0sec, Per = \
    bls_durations[nn_], bls_durations[mm_], bls_t0s[nn_] + tt_ref_, bls_t0s[mm_] + tt_ref_ + bls_periods[nn_], bls_periods[nn_]
    
    importlib.reload(helper_codes)
    ecosOm, esinOm, eccB, omegaB = helper_codes.ecosOm_esinOm(dur_prim, dur_sec, T0prim, T0sec, Per)
    print('Phase secondary =', "%0.2f" % ((-T0prim + T0sec)/Per % 1))
    print('ecos(omega) =', "%0.2f" % ecosOm, '; esin(omega) =', "%0.2f" % esinOm, \
          '; e =', "%0.2f" % eccB, '; omega =', "%0.2f" % (360+np.degrees(omegaB)))

In [ ]:
do_skyview = 'n'
if do_skyview == 'y':
    tpf_lk = lk.search_tesscut('TIC ' + str(int(TIC)), sector=hdr_["SECTOR"]).download_all(cutout_size = (9,9))
#    tpf_lk = lk.search_tesscut('TIC ' + str(int(172900988)), sector=21).download_all(cutout_size = (9,9))  
    tpf_lk[0].interact()#notebook_url='localhost:8888')

In [ ]:
if do_skyview == 'y':
    tpf_lk[0][500:].interact_sky(magnitude_limit = 21)#, notebook_url='localhost:8889')

In [ ]:
importlib.reload(helper_codes)
do_plot = 'y'
blend_mask_ = helper_codes.deblend(bls_periods, bls_durations, transit_mask_, tt_, tt_ref_, ff_, do_plot)

In [ ]:
remove_blends_ = input('Remove blends (default = n)? ')    
if len(remove_blends_) == 0:
        remove_blends_ = 'n'

if remove_blends_ == 'y':
    tt_bak = tt_
    tt_ = tt_[~blend_mask_]
    ff_ = ff_[~blend_mask_]
#    cube_clean = cube_clean[~blend_mask_]

In [ ]:
importlib.reload(helper_codes)
plot_folded = helper_codes.plot_phase_folded(TIC,tt_,tt_ref_,ff_,bls_periods, bls_t0s, bls_durations, bls_depths)

In [ ]:
#import statistics
#np.sqrt(statistics.variance(ff_sorted[np.logical_and(phase_sorted > -0.1, phase_sorted < -0.075)]))
#np.std(ff_sorted[np.logical_and(phase_sorted > -0.05, phase_sorted < 0.05)])

In [ ]:
if remove_blends_ == 'n':
    nn_ = 0
    fig = plt.figure(figsize=(20,3))
    plt.plot(tt_[~transit_mask_[nn_,:]]+tt_ref_, ff_[~transit_mask_[nn_,:]], 'k.-')
    plt.plot(tt_[transit_mask_[nn_,:]]+tt_ref_, ff_[transit_mask_[nn_,:]], 'ro')
    plt.xlim(tt_[0]+tt_ref_ - 0.*45, tt_[0]+tt_ref_+25.)
#    plt.ylim(0.9, 1.05)

In [ ]:
print('BLS Periods = ', bls_periods)
which_per = input('Which period (default = 0)? ') 
if len(which_per) == 0:
    which_per = 0
which_per = int(which_per)
        
for nn_ in range(which_per,which_per+1):#n_planets_):

    fn_ = path_ + str(TIC) + '_N' + str(nn_) +'_Per' + "%0.1f" % (bls_periods[nn_]) + \
    '_T' + "%0.3f" % (bls_t0s[nn_]+tt_ref_) + '_idx_bad_.npy'
    
#    nn_ = 0
    do_plot_ = 'y'
    importlib.reload(helper_codes_EBs)
    result = helper_codes_EBs.calculate_eclipse_times_test(np.asarray(tt_), tt_ref_, np.asarray(ff_), which_per, \
                                                  bls_periods, bls_durations, bls_t0s, bls_depths, do_plot_)

    cycle, \
    etv_trap, dur_trap, dep_trap, \
    etv_gg, etv_A_gg_unc, dur_gg, dep_gg, local_slope_gg, \
    etv_gauss, dur_gauss, dep_gauss = \
    result['cycle_A'], \
    result['etv_A_trap'], result['dur_A_trap'], result['dep_A_trap'], \
    result['etv_A_gg'], result['etv_A_gg_unc'], result['dur_A_gg'], result['dep_A_gg'], result['local_slope_gg'], \
    result['etv_A_gauss'], result['dur_A_gauss'], result['dep_A_gauss']
    
#    import os
#    if os.path.exists(fn_) == False:    
#        idx_bad_ = np.asarray((0))#,3,7,20,21,22,25,26,28,29))#0,11,33,41,45,))#13,14))#   
#        np.save(fn_, idx_bad_)
#    else:
#        idx_bad_ = np.load(fn_)
    
#    idx_bad_ = np.asarray((8,))
    idx_all_ = np.ones((len(cycle))).astype(bool)#np.asarray(list(range(0,10,1)))
    idx_bad_ = input('Bad eclipses? ')
    
    if len(idx_bad_) == 0:
        idx_bad_ = []
    else:
        idx_bad_ = np.asarray(idx_bad_.split (",")).astype(int)
        idx_all_[idx_bad_] = False
        
    cycle_A_, etv_trap_, etv_gg_, etv_A_gg_unc_, dep_gg_, etv_gauss_ = \
    cycle[idx_all_], etv_trap[idx_all_], etv_gg[idx_all_], etv_A_gg_unc[idx_all_], \
    dep_gg[idx_all_], etv_gauss[idx_all_]

    try:
        coef_trap, cov_trap = np.polyfit(cycle_A_, etv_trap_, deg = 1, cov=True)
        coef_gg, cov_gg = np.polyfit(cycle_A_, etv_gg_, deg = 1, cov=True)
        coef_gauss, cov_gauss = np.polyfit(cycle_A_, etv_gauss_, deg = 1, cov=True)        
    except:
        coef_trap = np.polyfit(cycle_A_, etv_trap_, deg = 1)
        coef_gg = np.polyfit(cycle_A_, etv_gg_, deg = 1)
        coef_gauss = np.polyfit(cycle_A_, etv_gauss_, deg = 1) 
        cov_trap, cov_gg, cov_gauss = [[0.,0.], [0.,0.]], [[0.,0.], [0.,0.]],[[0.,0.], [0.,0.]]
        
    poly1d_fn_trap = np.poly1d(coef_trap) 
    per_A_trap = poly1d_fn_trap.c[0]
    
    poly1d_fn_gg = np.poly1d(coef_gg)     
    per_A_gg = poly1d_fn_gg.c[0]
        
    poly1d_fn_gauss = np.poly1d(coef_gauss) 
    per_A_gauss = poly1d_fn_gauss.c[0]

In [ ]:
# coef_gg, cov_gg = np.polyfit(cycle_A_, etv_gg_, deg = 1, cov = True)
# print(coef_gg[0], per_A_gg, np.sqrt(cov_gg[0][0]))

In [ ]:
###### fn_ = path_ + '392229331_P1.8_T1819.08_N0_idx_bad_.npy'#'392229331_P1.8_T1819.08_N0_idx_bad_.npy'
#idx_bad_ = np.load(fn_)
#idx_bad_#, fn_

In [ ]:
fig = plt.figure(figsize=(20,14))
        
grid = plt.GridSpec(3, 2, wspace=0.3, hspace=0.3)
#        
#
#
# per_A_gg = 2.*per_A_gg
#etv_gg_, per_A_gg = etv_gauss_, per_A_gauss

plt.subplot(grid[0, :])
phase_ = (tt_ - (etv_gg_[-1]-tt_ref_))/per_A_gg % 1
shift_ = phase_ > 0.6
phase_[shift_] = phase_[shift_] - 1.
sorted_ = phase_.argsort()
phase_sorted_, ff_sorted_ = phase_[sorted_], ff_[sorted_]

plt.plot(phase_sorted_, ff_sorted_, 'k.')
plt.title("TIC "  + str(TIC) + "; P" + str(nn_+1) + " = " + "%0.6f" % per_A_gg + \
              "; Dur" + str(nn_+1) + " = " + "%0.1f" % (np.nanmedian(dur_trap)) + " hr"  + \
              "; Dep" + str(nn_+1) + " = " + "%0.f" % (1e3*np.nanmedian(dep_gg)) + ' ppt', fontsize = 18)# + \
plt.xlim(-0.4, 0.6)
#plt.xlim(-0.1, 0.1)
plt.axvline(0.5, color = 'r')
#plt.axhline(1.-np.nanmedian(dep_gg_), color = 'k', ls = '--')
#plt.ylim(0.85, 1.05)
#plt.xlim(-0.15,0.15)
#
#
#
plt.subplot(grid[2, :])
fit_trap = poly1d_fn_trap.c[0]*np.linspace(0+0*cycle_A_[0],cycle_A_[-1],int(cycle_A_[-1])+1) + poly1d_fn_trap.c[1]
fit_gg = poly1d_fn_gg.c[0]*np.linspace(0+0*cycle_A_[0],cycle_A_[-1],int(cycle_A_[-1])+1) + poly1d_fn_gg.c[1]
fit_gauss=poly1d_fn_gauss.c[0]*np.linspace(0+0*cycle_A_[0],cycle_A_[-1],int(cycle_A_[-1])+1) + poly1d_fn_gauss.c[1]

plt.plot(1.*57e3 + etv_gg_, cycle_A_, 'ro', mfc = 'None', ms = 16)
plt.plot(1.*57e3 + fit_trap, np.linspace(0+0*cycle_A_[0],cycle_A_[-1],int(cycle_A_[-1])+1), 'k--')
plt.plot(1.*57e3 + fit_gg, np.linspace(0+0*cycle_A_[0],cycle_A_[-1],int(cycle_A_[-1])+1), 'k--')
plt.plot(1.*57e3 + fit_gauss, np.linspace(0+0*cycle_A_[0],cycle_A_[-1],int(cycle_A_[-1])+1), 'k--')
#plt.title('Binary A; P = ' + str("%0.6f" % per_A_gg), fontsize = 18)# + '; Secondary Eclipses', fontsize = 18)
plt.ylabel('Cycle #', fontsize = 18)
plt.xlim(-5 + min(1.*57e3 + fit_gg), 5. + max(1.*57e3 + fit_gg))
idx_ = cycle_A_.astype(int)
#
#
plt.subplot(grid[1, :])
plt.plot(1.*57e3 + etv_gg_, etv_gg_ - fit_gg[idx_], 'rs', ms = 16, mfc = 'None', label = 'Gen Gauss Fit')
#plt.errorbar(1.*57e3 + etv_gg_, etv_gg_ - fit_gg[idx_], yerr = etv_A_gg_unc_, fmt='ro', \
#             ms = 16, mfc = 'None', label = 'Gen Gauss Fit')
#plt.plot(1.*57e3 + etv_trap_, etv_trap_ - fit_trap[idx_], 'b*', ms = 16, mfc = 'None', label = 'Trap Fit')
#plt.plot(1.*57e3 + etv_gauss_, etv_gauss_ - fit_gauss[idx_], 'c*', ms = 16, mfc = 'None', label = 'Gauss Fit')
#plt.ylim(-0.01, 0.01)
plt.ylabel('ETV [days]', fontsize = 18)
plt.xlabel('Time [BJD - 2,400,000] days]', fontsize = 18)
plt.legend(fontsize = 14)
plt.xlim(-5 + min(1.*57e3 + fit_gg), 5. + max(1.*57e3 + fit_gg))
#plt.xlim(min(1.*57e3 + fit_gg), 59e3)
#plt.xlim(58.94e3,58.95e3)
#plt.xlim(1.*57e3 + 1460, 1.*57e3 + 1550)

#plt.subplot(grid[2, 1])
#plt.plot(0.*57e3 + etv_trap_, etv_gg - fit_gg[idx_], 'mo', ms = 16, mfc = 'None', label = 'Trap Fit')
#plt.plot(1.*57e3 + etv_trap_, etv_gauss - fit_gauss[idx_], 'co', ms = 16, mfc = 'None', label = 'Gauss Fit')
#plt.plot(1.*57e3 + etv_trap_, etv_trap - fit_trap[idx_], 'r*', ms = 10, mfc = 'r', label = 'Gen Gauss Fit')
#plt.ylim(-0.1, 0.1)
#plt.ylabel('ETV [days]', fontsize = 18)
#plt.legend(fontsize = 14)
#plt.xlim(-5 + min(0.*57e3 + fit_), 5. + max(0.*57e3 + fit_))
#plt.xlim(1.*57e3 + 2100, 1.*57e3 + 2200)

if which_per == 0:
    tp, etvp, times_prim, slopep = 1.*57e3+etv_gg_, etv_gg_ - fit_gg[idx_], etv_gg_, local_slope_gg[idx_all_]
elif which_per == 1:
    ts, etvs, times_sec, slopes = 1.*57e3+etv_gg_, etv_gg_ - fit_gg[idx_], etv_gg_, local_slope_gg[idx_all_]
elif which_per > 1:
    tx, etvx, times_x, slopex = 1.*57e3+etv_gg_, etv_gg_ - fit_gg[idx_], etv_gg_, local_slope_gg[idx_all_]

#which_ecl_ = 'prim' if nn_ == 0 else 'sec'
plt.savefig('/Users/vkostov/Desktop/TIC_' + str(TIC) + '_eclipse_times_' + str(which_per) + '.png')

print(TIC, etv_gg_[-1], per_A_gg, "%0.f" % (np.nanmedian(dep_gg_*1e3)))

In [ ]:
#np.set_printoptions(precision=4)
#bls_periods

In [ ]:
fig = plt.figure(figsize=(15,3))
tmp_per_ = bls_periods[nn_]#13.620154#13.619800#13.620154
    
#    print(tmp_per_)
tmp_per_ = per_A_gg#8.858704#
 
phase_ = (tt_ - (etv_gg_[-1] - tt_ref_))/tmp_per_ % 1

shift_ = phase_ > 0.6
phase_[shift_] = phase_[shift_] - 1.        
sorted_ = phase_.argsort()
phase_sorted, ff_sorted = phase_[sorted_], ff_[sorted_]
        
#    plt.plot(phase_, ff_, 'k.')
plt.plot(phase_sorted, ff_sorted, 'k.', alpha = 0.5)

importlib.reload(helper_codes)
do_plot = 'n'
trap_fit, gauss_fit, depth, duration, param_uncertainties = \
helper_codes.fit_eclipse(phase_, ff_, do_plot, bls_periods, bls_depths[nn_], nn_)    
center_ = np.where((phase_sorted > -0.1) & (phase_sorted < 0.1))
plt.plot(phase_sorted[center_], -0.00 + trap_fit[center_], 'r', alpha = 1, lw = 1)
# plt.plot(phase_sorted[center_], 0.0 + gauss_fit[center_], 'c', alpha = 0.75)
plt.xlim(-0.4,0.6)
# plt.xlim(-0.05, 0.05)
plt.axvline(0.5, linestyle = '--', alpha = 0.5)
# plt.ylim(0.98, 1.02)
plt.title('TIC ' + str(TIC) + '; P = ' + "%0.5f" % tmp_per_)

print(TIC, "%0.5f" % tmp_per_, "%0.5f" % np.sqrt(np.diag(cov_gg))[0], "%0.4f" % etv_gg_[-1], \
          "%0.4f" % np.sqrt(np.diag(cov_gg))[1], "%0.1f" % (duration*24), \
          "%0.1f" % (np.nanmedian(dur_trap[idx_all_])), "%0.2f" % np.std(dur_trap[idx_all_]), \
          "%0.f" % (np.nanmedian(dep_gg_*1e3)))

In [ ]:
bls_periods[which_per] = per_A_gg
bls_t0s[which_per] = etv_gg_[0] - tt_ref_
bls_durations[which_per] = duration
bls_depths[which_per] = np.nanmedian(dep_gg_)

In [ ]:
plot_etvs_ = False
if plot_etvs_:

    fig = plt.figure(figsize=(20,10))
    plt.rcParams.update({'font.size': 22})
    
    grid = plt.GridSpec(2, 2, wspace=0.3, hspace=0.1)
    
    nn_ = 0
    ax0 = plt.subplot(grid[0, 0])    
    phase_ = (tt_ - (bls_t0s[nn_]-0.*tt_ref_))/bls_periods[nn_] % 1
    shift_ = phase_ > 0.6
    phase_[shift_] = phase_[shift_] - 1.
    sorted_ = phase_.argsort()
    phase_sorted_, ff_sorted_ = phase_[sorted_], ff_[sorted_]
    ax0.plot(phase_sorted_, ff_sorted_, 'k.')
    plt.title("TIC "  + str(TIC) + "; PA = " + "%0.6f" % bls_periods[nn_])# +  "; P2 = " + "%0.6f" % bls_periods[1])
    plt.xlim(-0.4, 0.6)
    plt.xlim(-0.1, 0.1)
    
    nn_ = 1
    ax1 = plt.subplot(grid[0, 1])    
    phase_ = (tt_ - (bls_t0s[nn_]-0.*tt_ref_))/bls_periods[nn_] % 1
    shift_ = phase_ > 0.6
    phase_[shift_] = phase_[shift_] - 1.
    sorted_ = phase_.argsort()
    phase_sorted_, ff_sorted_ = phase_[sorted_], ff_[sorted_]
    ax1.plot(phase_sorted_, ff_sorted_, 'k.')
    plt.title("TIC "  + str(TIC) + "; PB = " + "%0.6f" % bls_periods[nn_])# +  "; P2 = " + "%0.6f" % bls_periods[1])
    #plt.ylim(0.98, 1.01)
    plt.xlim(-0.4, 0.6)
    plt.xlim(-0.1, 0.1)
    
    
    ax2 = plt.subplot(grid[1, :]) 
    ax2.plot(tp-55e3, etvp*24*60, 'ro', ms = 18, label = 'PAprim', mfc = 'None')
    ax2.plot(ts-55e3, etvs*24*60, 'b.', ms = 10, label = 'PAsec', mfc = 'b')
    #ax2.plot(tx-55e3, etvx*24*60, 'g*', ms = 18, label = 'PBprim', mfc = 'None')
    #plt.axvline(59483.85, color = 'k', ls = '--', label = 'tertiary')
    plt.legend()
    plt.ylabel('ETV [min]', fontsize = 18)
    plt.xlabel('Time [BJD - 2,450,000] days]', fontsize = 18)
    #plt.title('Detrended')
    #plt.xlim(4300, 4350)
    #plt.ylim(2, 10)
    
    plt.tight_layout()

In [ ]:
#for ii in range(len(tp)):
#    print("%0.6f" % tp[ii], ii+0., "%0.6f" % etvp[ii])

In [ ]:
plot_local_slope = 'n'
if plot_local_slope == 'y':
    fig = plt.figure(figsize=(25,17))
    plt.rcParams.update({'font.size': 22})

    grid = plt.GridSpec(2, 2, wspace=0.3, hspace=0.3)

    ax1 = plt.subplot(grid[0, :])    
#    ax1 = plt.subplot(3,1,1)
#plt.plot(1.*57e3 + etv_gg_, etv_gg_ - fit_gg[idx_], 'ro', ms = 16, mfc = 'None', label = 'Gen Gauss Fit')
    ax1.plot(tp-57e3, etvp, 'ro', ms = 16, mfc = 'None', label = 'Primary')
    ax1.plot(ts-57e3, etvs, 'b*', ms = 16, mfc = 'None', label = 'Secondary')
    plt.legend()
    plt.xlabel('BJD - 2,457,000')
    plt.ylabel('ETVs (days)')
#    plt.ylim(-0.004, 0.004)

    ax2 = plt.subplot(grid[1, 0])#plt.subplot(3,1,2)
    ax2.plot(slopep, etvp*24*60, 'ro', ms = 16, mfc = 'None')
    tmp_xx_ = np.linspace(min(slopep), max(slopep), 100)
    p2 = np.poly1d(np.polyfit(slopep, etvp*24*60, 1))
    ax2.plot(tmp_xx_, p2(tmp_xx_), 'k--', label = 'Linear fit primary')
#    plt.xlabel('Local light curve slope')
    plt.ylabel('ETVs (min)')
    plt.legend()

    ax3 = plt.subplot(grid[1, 1])#plt.subplot(3,1,3)
    idx_bad_secondary = np.ones(len(etvs)).astype(bool)
    idx_bad_secondary[etvs.argmax()] = False
    ax3.plot(slopes[idx_bad_secondary], etvs[idx_bad_secondary]*24*60, 'b*', ms = 16, mfc = 'None')
    tmp_xx_ = np.linspace(min(slopes[idx_bad_secondary]), max(slopes[idx_bad_secondary]), 100)
    p3 = np.poly1d(np.polyfit(slopes[idx_bad_secondary], etvs[idx_bad_secondary]*24*60, 1))
    ax3.plot(tmp_xx_,p3(tmp_xx_), 'k--', label = 'Linear fit secondary')
    plt.xlabel('Local light curve slope')
#    plt.ylabel('ETVs (days)')
    plt.legend()


In [ ]:
plot_etvs_ = 'n'
if plot_etvs_ == 'y':
    
    which_eclipses_ = 0

    #t_prim, etv_prim = 1.*57e3 + etv_gauss_, etv_gauss_ - fit_gauss[idx_]
    #t_sec, etv_sec = 1.*57e3 + etv_gauss_, etv_gauss_ - fit_gauss[idx_]
    fig = plt.figure(figsize=(25,12))
    plt.rcParams.update({'font.size': 20})
    #fig, ax = plt.subplots(figsize=(25,15))

    ax1 = plt.subplot(2,1,2) 
    ax1.plot(tp-0.*57e3, 24*60*etvp-0.*0.0015, 'ro', mfc = 'None', ms = 22, label = 'Primary')
    ax1.plot(ts-0.*57e3, 24*60*etvs+0.*0.0015, 'b.', ms = 12, mfc = "b", label = 'Secondary')
    plt.legend()#fontsize = 16)
    plt.ylabel('ETV [min]', fontsize = 28)
    plt.xlabel('Time [BJD - 2,400,000] days]', fontsize = 28)
#    plt.ylim(-0.004,0.004)

    cpoc = np.mean(bls_periods)

    ax2 = plt.subplot(2,1,1)     
    phase_ = (tt_ - (bls_t0s[which_eclipses_]-0.*tt_ref_))/bls_periods[which_eclipses_] % 1
    shift_ = phase_ > 0.6
    phase_[shift_] = phase_[shift_] - 1.
    sorted_ = phase_.argsort()
    phase_sorted_, ff_sorted_ = phase_[sorted_], ff_[sorted_]

    ax2.plot(phase_sorted_, ff_sorted_, 'k.')
    plt.title("TIC "  + str(TIC) + "; P1 = " + "%0.6f" % bls_periods[0] +  "; P2 = " + "%0.6f" % bls_periods[1])
    
    
    bls_periods[which_per]
#    plt.title("TIC "  + str(TIC) + "; P" + str(nn_+1) + "= " + "%0.6f" % per_A_gg + \
#              "; Dur 1 = " + "%0.2f" % (np.nanmedian(dur_trap)) + \
#              "; Dep 1 = " + "%0.f" % (1e3*bls_depths[0]) + \
#              "; Dep 2 = " + "%0.f" % (1e3*bls_depths[1]), fontsize = 18)# + \
    plt.xlim(-0.4, 0.6)
#    plt.axvline(0.5, color = 'k', alpha = 0.5)
#    plt.axhline(1.-bls_depths[0], color = 'r')
#    plt.axhline(1.-bls_depths[1], color = 'b')
#    plt.axvline(0.5, color = 'r', ls = '--')
    
#    ax2.plot(per, chisq, 'k')
#    ax2.tick_params(labelcolor='k', labelsize='medium', width=2, length = 13)
#    ax2.tick_params(axis='x', which='minor', width=2, length = 10)
#    ax2.set_xscale('log')
#    for ii in range(1,8):
#        plt.axhline(ii, color = 'k', ls = 'dotted', lw = 1)
#    plt.ylabel(r"${\rm \chi^{2}_{\nu}}$")#"Trial Period (days)")
#    plt.xlabel("Trial Period (days)")

In [ ]:
# CHECK ETVs WITH LOMB-SCARGLE
do_etvs_LS = 'n'
if do_etvs_LS == 'y':
    
    tmp_etvs_ = etvp
    tmp_time_etvs_ = tp

    from astropy.timeseries import LombScargle
    frequency, power = LombScargle(tmp_time_etvs_, \
                               tmp_etvs_).autopower(minimum_frequency=1./300., \
                                                  maximum_frequency = 1./20,samples_per_peak=50)

    fig = plt.figure(figsize=(20,12))

    grid = plt.GridSpec(3, 2, wspace=0.1, hspace=0.3)
    
    plt.subplot(grid[0, :])
    plt.plot(phase_sorted_, ff_sorted_, 'k.')
    plt.title("TIC "  + str(TIC) + "; P3 = " + "%0.6f" % per_A_gg, fontsize = 18)# + \
    plt.xlim(-0.4, 0.6)

    plt.subplot(grid[1, :])
    plt.plot(tp-57e3, 24*60*etvp-0.*0.0015, 'ro', mfc = 'None', ms = 16, label = 'Primary')
    # plt.plot(ts-57e3, 24*60*etvs+0.*0.0015, 'b.', ms = 12, mfc = "b", label = 'Secondary')
    plt.legend()#fontsize = 16)
    plt.ylabel('ETV [min]', fontsize = 28)
    plt.axvline(1704, c = 'k', ls = '--')
    plt.axvline(2806, c = 'k', ls = '--')
    #plt.xlabel('Time [BJD - 2,400,000] days]', fontsize = 28)
    #plt.plot(tmp_time_etvs_, tmp_etvs_, 'co', ms = 16, mfc = 'None')

    plt.subplot(grid[2, 0])
    plt.plot(1./frequency, power, 'k')
    plt.xlabel('Period [days]', fontsize = 18)
    plt.ylabel('Lomb-Scargle', fontsize = 18)

    plt.subplot(grid[2, 1])
    best_frequency = frequency[np.argmax(power)]
    t_fit = np.linspace(0, 1)
    plt.plot(tmp_time_etvs_/(1/best_frequency) % 1, tmp_etvs_, 'ko')
    plt.xlabel('Phase outer orbit', fontsize = 18)
    plt.title('Pouter = ' + str(np.round(1/best_frequency,2)) + ' days', fontsize = 18)
    
#    plt.savefig('/Users/vkostov/Desktop/TIC_' + str(TIC) + '_ETVs_' + str(which_per) + '_LS.png')

In [ ]:
#both = set(list(tt_AA)).intersection(list(tt_BB))
#indices_A = [list(tt_AA).index(x) for x in both]
#indices_B = [list(tt_BB).index(x) for x in both]
#print(both)

In [ ]:
def moving_average(x, w):
    return np.convolve(x, np.ones(w), 'valid') / w
#lc_SC = lk.search_lightcurvefile('TIC ' + str(TIC), sector = 33).download()
#tt, flux_SAP, flux_PDC = lc_SC.time, lc_SC.SAP_FLUX.flux, lc_SC.PDCSAP_FLUX.flux
#SAP_10 = moving_average(flux_SAP/np.nanmean(flux_SAP), 100)

In [ ]:
fig = plt.figure(figsize=(20,12))
mm = np.ones(len(ff_), dtype=bool)

for nn_ in range(2):

    fig = plt.figure(figsize=(20,2.5))
#    tmp_per_ = bls_periods[nn_]#13.620154#13.619800#13.620154
    plt.plot(tt_ + 1.*tt_ref_, ff_, 'k.-')
    mm = np.ones(len(ff_), dtype=bool)
    mm[(tt_ + 1.*tt_ref_ >= (1.*tt_ref_ + bls_t0s[which_per] + nn_*bls_periods[which_per]) - \
        1.*bls_durations[which_per]) & \
       (tt_ + 1.*tt_ref_ <= (1.*tt_ref_ + bls_t0s[which_per] + nn_*bls_periods[which_per]) + \
        1.*bls_durations[which_per])] = False    
#    mm_ = np.hstack((mm,mm))
    plt.plot(tt_[~mm] + 1.*tt_ref_, ff_[~mm], 'ro')
    plt.title("TIC "  + str(TIC) + "; Period " + str(which_per+1) + "= " + str(np.round(bls_periods[which_per],5)) + \
              "; Duration 1 = " + str(np.round(24.*bls_durations[0],2)), fontsize = 18)# + \

    plt.xlim(1.*tt_ref_ + bls_t0s[which_per] + nn_*tmp_per_- 40.*bls_durations[which_per], \
             1.*tt_ref_ + bls_t0s[which_per] + (nn_)*tmp_per_+ 40.*bls_durations[which_per])
    

In [ ]:
x, y = tt_ + tt_ref_, np.ones(len(ff_))

per_, x0, dur_, dep_ = bls_periods[which_per], bls_t0s[which_per] + tt_ref_, \
bls_durations[which_per], bls_depths[which_per]
n_periods_ = int(np.ceil((x[-1] - x[0])/per_))
nn_ = np.linspace(0,n_periods_-1,n_periods_)

mm = np.ones(len(y), dtype=bool)

for ii in range(n_periods_):
    mm[(x - 0.*np.min(x) >= (x0 + 0.*tt_ref_ + ii*per_) - 0.5*dur_) \
      & (x - 0.*np.min(x) <= (x0 + 0.*tt_ref_ + ii*per_) + 0.5*dur_)] = False

y[~mm] = 1.-dep_

fig = plt.figure(figsize=(20,3))
plt.plot(tt_+1.*tt_ref_, ff_, 'k.-', lw = 1)
plt.plot(x, 0.01 + y, 'r', lw = 2)
plt.xlim(min(x), max(x))

#plt.savefig('/Users/vkostov/Desktop/TIC_' + str(TIC) + '_BLS_periods.png')

In [ ]:
deblend = 'n'
if deblend == 'y':
    print(cube.shape)
    tt_, ff_, cube = tt_[~blend_mask_], ff_[~blend_mask_], cube[~blend_mask_]
#    cube = cube[~blend_mask_]
    print(cube.shape)

In [ ]:
def sub_cube(cube, sub_cube_size):
    cube_size = cube.shape[-1]
    sub_cube_xmin = int(0.5*(cube_size - sub_cube_size))
    sub_cube_xmax = sub_cube_xmin + sub_cube_size
    return cube[:,sub_cube_xmin:sub_cube_xmax,sub_cube_xmin:sub_cube_xmax], sub_cube_xmin

In [ ]:
def Multisector_centroids(which_sector, arr_, bad_times_, remove_blends_, sub_cube_size, compare_DIFF_to_):
    
    diffC_PSF, diffR_PSF = [], []
    diffC_PRF, diffR_PRF = [], []
    
    for zz in which_sector:

        do_plot_ = 'n'
        tt, rr_, cc_, pca_, qq_, cube_, TIC, sector_, camera_, ccd_, tpf_x, tpf_y, hdr_ = \
        helper_codes.get_data_(arr_, [zz], do_plot_)
#
#
        tt_clean, ff_clean, cube_clean = tt[~qq_.astype(bool)], cc_[~qq_.astype(bool)], cube_[~qq_.astype(bool)]
        tt_ref_clean = min(tt_clean)
#        tt_clean -= tt_ref_clean
#
#
        custom_bad_times = np.ones(len(ff_clean), dtype=bool)
        for bb in range(len(bad_times_)):
            custom_bad_times[(tt_clean + tt_ref_clean >= bad_times_[bb,0]) & \
                             (tt_clean + tt_ref_clean <= bad_times_[bb,1])] = False

        tt_clean = tt_clean[custom_bad_times]
        ff_clean = ff_clean[custom_bad_times]
        cube_clean = cube_clean[custom_bad_times]
#
#
        deblend_ = np.ones(len(tt_clean), dtype=bool)
        if remove_blends_ == 'y':    
            interesect_, x_ind, y_ind = np.intersect1d(tt_clean + 0.*tt_ref_clean, tt_bak[blend_mask_] + tt_ref_, \
                                               return_indices = True)            
            if len(interesect_) > 0:
                deblend_[x_ind] = False

        tt_ = tt_clean[deblend_]
        ff_ = ff_clean[deblend_]
        cube = cube_clean[deblend_]
#
#
        if zz == which_sector[0]:
            print('BLS Periods = ', bls_periods)
            which_period = input('Which period (default = 0)? ') 
            if len(which_period) == 0:
                which_period = 0
            which_period = int(which_period)
#
#       
        try:
#        if 1 == 1:        
            xc_PSF, yc_PSF, xc_PSF_OOT, yc_PSF_OOT, xc_PRF, yc_PRF, diff_cube_, sub_cube_xmin, res, clip = \
            helper_codes.Per_sector_centroids(TIC, \
                                          which_period, \
                                          tt_+0.*tt_ref_clean, ff_, tt_ref_clean, tt_ref_, \
                                          bls_periods, bls_t0s+1.*tt_ref_, bls_depths, bls_durations, \
                                          cube, sub_cube_size, \
                                          sector_, camera_, ccd_, tpf_x, tpf_y, hdr_)   
#            
#            
            show_cent_fits = 'y'
            if show_cent_fits == 'y':
                for ii in range(diff_cube_.shape[-1]):
                    fig = plt.figure(figsize=(3,3))
                    plt.imshow(diff_cube_[:,:,ii], origin = 'lower', \
                           extent = [0,diff_cube_.shape[0],0,diff_cube_.shape[1]])
                    plt.plot(xc_PRF[ii]-1*sub_cube_xmin, yc_PRF[ii]-1*sub_cube_xmin, 'ro', ms = 10)
                    plt.plot(xc_PSF[ii]-0*sub_cube_xmin, yc_PSF[ii]-0*sub_cube_xmin, 'kv', ms = 14, mfc = 'None')
                    plt.text(sub_cube_size-2, sub_cube_size-2, 'T' + str(ii), c = 'w', fontsize = 16)
#                plt.text(sub_cube_size+2, sub_cube_size+2, 'S' + str(zz), c = 'w', fontsize = 16)
                    plt.xticks([])
                    plt.yticks([])
                    plt.show()
#
#
            transits = getIngressEgressCadences(tt_+0.*tt_ref_clean, bls_periods[which_period], \
                                            bls_t0s[which_period] + 1.*tt_ref_, bls_durations[which_period])
            bad_transit_ = np.zeros(len(transits), dtype = bool)
        
            for jj in range(len(transits)):
                cin = transits[jj]
                key = 'transit-%04i' %(jj)
        
                no_data_ = np.nanmean(ff_[cin[0]:cin[1]])

                if (res[key]['errorCode'] > 7) or (np.isnan(no_data_) == True):
                    bad_transit_[jj] = True
                    print('Transit #', str(jj) + ' Bad')
            
            transits = transits[~bad_transit_]
            
            try:
                plot_transits = 'y'
                if plot_transits == 'y':
                    fig = plt.figure(figsize=(20,5))
                    plt.plot(tt_+0.*tt_ref_clean, ff_, 'k')
                    for pp in range(transits.shape[0]):
                        plt.axvspan(tt_[transits[pp,0]], tt_[transits[pp,1]], color = 'r', lw = 1, alpha = 0.1)
                plt.show()
            except:
                temp_var = 999.
            
            bad_transit_idx = input('Bad Transits (default = None)? ') 
            if len(bad_transit_idx) > 0:
                bad_transit_idx = [int(i) for i in bad_transit_idx.split(',')]
                bad_transit_[bad_transit_idx] = True
#
#            
            TIC_x, TIC_y = clip.serve.param.TIC_x, clip.serve.param.TIC_y  

            # print(-TIC_x + xc_PSF[~bad_transit_] + 1.*sub_cube_xmin, -xc_PSF_OOT + xc_PSF[~bad_transit_])
    
            if zz == which_sector[0]:
                if compare_DIFF_to_ == 'TIC':
                    diffC_PSF, diffR_PSF = \
                    -TIC_x + xc_PSF[~bad_transit_] + 1.*sub_cube_xmin, -TIC_y + yc_PSF[~bad_transit_] + 1.*sub_cube_xmin                    
                elif compare_DIFF_to_ == 'OOT':
                    diffC_PSF, diffR_PSF = \
                    -xc_PSF_OOT[~bad_transit_] + xc_PSF[~bad_transit_], -yc_PSF_OOT[~bad_transit_] + yc_PSF[~bad_transit_]

                diffC_PRF, diffR_PRF = \
                    -TIC_x + xc_PRF[~bad_transit_], -TIC_y + yc_PRF[~bad_transit_]  
                    
            else:
                if compare_DIFF_to_ == 'TIC':
                    diffC_PSF = np.hstack((diffC_PSF, -TIC_x + xc_PSF[~bad_transit_] + 1.*sub_cube_xmin))
                    diffR_PSF = np.hstack((diffR_PSF, -TIC_y + yc_PSF[~bad_transit_] + 1.*sub_cube_xmin))
                elif compare_DIFF_to_ == 'OOT':
                    diffC_PSF = np.hstack((diffC_PSF, -xc_PSF_OOT + xc_PSF[~bad_transit_]))
                    diffR_PSF = np.hstack((diffR_PSF, -yc_PSF_OOT + yc_PSF[~bad_transit_]))
                            
                diffC_PRF = np.hstack((diffC_PRF, -TIC_x + xc_PRF[~bad_transit_]))
                diffR_PRF = np.hstack((diffR_PRF, -TIC_y + yc_PRF[~bad_transit_]))
#        
#        
        except:
#        else:
            diffC_PSF, diffR_PSF = np.hstack((diffC_PSF,999.)), np.hstack((diffR_PSF,999.))
            diffC_PRF, diffR_PRF = np.hstack((diffC_PRF,999.)), np.hstack((diffR_PRF,999.))
            print(zz, "BAD TRANSIT")
               
    return diffC_PSF, diffR_PSF, diffC_PRF, diffR_PRF, cube, diff_cube_, sub_cube_xmin, which_period, TIC_x, TIC_y

In [ ]:
#bls_durations[1] = 0.15

In [ ]:
# importlib.reload(helper_codes)
from dave.tessPipeline import pertransitcentroids
importlib.reload(pertransitcentroids)

#which_sector_ = [1]
compare_DIFF_to_ = 'TIC'#'OOT'#

sub_cube_size = 7
#which_sector_tmp = np.asarray([0])
diffC_PSF, diffR_PSF, diffC_PRF, diffR_PRF, cube, diff_cube_, sub_cube_xmin, which_period, TIC_x, TIC_y = \
Multisector_centroids(which_sector, arr_, bad_times_, remove_blends_, sub_cube_size, compare_DIFF_to_)

In [ ]:
# dict_cent = dict()
#if dict_cent:
if 'dict_cent' in locals() or 'dict_cent' in globals():
    pass
else:
    dict_cent = dict()
    
dict_cent['TIC ' + str(TIC) + '; P = ' + str("%0.1f" % bls_periods[which_period]) + ' d; TIC_x [px]'] = TIC_x
dict_cent['TIC ' + str(TIC) + '; P = ' + str("%0.1f" % bls_periods[which_period]) + ' d; TIC_y [px]'] = TIC_y
dict_cent['TIC ' + str(TIC) + '; P = ' + str("%0.1f" % bls_periods[which_period]) + ' d; xc_PSF_1'] = diffC_PSF + TIC_x
dict_cent['TIC ' + str(TIC) + '; P = ' + str("%0.1f" % bls_periods[which_period]) + ' d; yc_PSF_1'] = diffR_PSF + TIC_y
# dict_cent['TIC ' + str(TIC) + '; P = ' + str("%0.3f" % bls_periods[which_period]) + ' d; diffC_PSF'] = diffC_PSF 
# dict_cent['TIC ' + str(TIC) + '; P = ' + str("%0.3f" % bls_periods[which_period]) + ' d; diffR_PSF'] = diffR_PSF

In [ ]:
# importlib.reload(helper_codes)
# TIC1, TIC2 = 39608572, 1352685042
# aa = helper_codes.plot_centroids_wrt_nearby_tics(dict_cent, bls_periods, TIC1, TIC2)

In [ ]:
# diffC_PSF[0] = diffC_PSF[1] - 0.0015
# diffR_PSF[0] = diffR_PSF[1] + 0.0015

In [ ]:
fig = plt.figure(figsize=(15,5))
#fig, ax = plt.subplots(figsize=(15, 5))

ax1 = plt.subplot(121)
diffC, diffR = diffC_PSF, diffR_PSF
bad_transit_ = np.zeros(len(diffC_PSF), dtype = bool)
bad_transit_idx = 0 + np.asarray((4))#4,5,10))#1,7,8,15,16,18,24,25,32,33,43,48))
cut__ = 1.5
# bad_transit_idx, = np.where((diffC_PSF > cut__) | (diffC_PSF < -cut__) | (diffR_PSF > cut__) | (diffR_PSF < -cut__))
# bad_transit_[bad_transit_idx] = True

for vv in range(3):
    covar.confidence_ellipse(diffC[~bad_transit_], diffR[~bad_transit_], ax1, vv+1, \
                             ec='k', fc = 'gray', alpha = 1/3/(vv+1))
#covar.plotErrorEllipse(diffC[~bad_transit_], diffR[~bad_transit_], color='#888888', ms=20, mfc = 'None')    

ax1.plot(diffC[~bad_transit_], diffR[~bad_transit_], 'ro', ms = 8)
ax1.plot(np.nanmedian(diffC[~bad_transit_]), np.nanmedian(diffR[~bad_transit_]), 'ro', ms = 26, mfc = 'None')
ax1.plot(0,0, 'k*', mfc = "None", ms = 36, mew = 2.)
#ax1.text(0+0.05, 0+0.05, 'TIC ' + str(TIC), fontsize = 16)
plt.xlabel(r"$\Delta$ Column (pixels)")
plt.ylabel(r"$\Delta$ Row (pixels)")
plt.axhline(0, color = 'k', alpha = 0.3, ls = 'dashed')
plt.axvline(0, color = 'k', alpha = 0.3, ls = 'dashed')

#plt.grid()
plt.title('TIC ' + str(TIC) + '; P = ' + "%.2f" % bls_periods[which_period], fontsize = 18)# + \
#          '; PSF Offset = ' + str("%0.2f" % (1.*(label_))) + ' px', fontsize = 18)
#plt.xlim(2*min(diffC), 2*max(diffC))
#plt.ylim(2*min(diffR), 2*max(diffR))

size_ = 1.
# plt.xlim(-size_, size_)
# plt.ylim(-size_, size_)
ax1.set_aspect('equal')


#plt.show()
#
#
#
ax2 = plt.subplot(122)
diffC, diffR = diffC_PRF, diffR_PRF 
bad_transit_ = np.zeros(len(diffC_PSF), dtype = bool)
bad_transit_idx = 0 + np.asarray((0))#4,5,10))#1,7,8,15,16,18,24,25,32,33,43,48))
cut__ = 0.25
bad_transit_idx, = np.where((diffC_PSF > cut__) | (diffC_PSF < -cut__) | (diffR_PSF > cut__) | (diffR_PSF < -cut__))
# bad_transit_[bad_transit_idx] = True
   
label_ = covar.computeProbabilityOfObservedOffset(diffC[~bad_transit_], diffR[~bad_transit_])[0]
label_ = np.sqrt(np.median(diffC[~bad_transit_])**2 + np.median(diffR[~bad_transit_])**2)

for vv in range(3):
    covar.confidence_ellipse(diffC[~bad_transit_], diffR[~bad_transit_], ax2, vv+1, \
                             ec='k', fc = 'gray', alpha = 1/3/(vv+1))
#covar.plotErrorEllipse(diffC[~bad_transit_], diffR[~bad_transit_], color='#888888', ms=20, mfc = 'None')    
    
ax2.plot(diffC[~bad_transit_], diffR[~bad_transit_], 'ro', ms = 8)
ax2.plot(np.nanmedian(diffC[~bad_transit_]), np.nanmedian(diffR[~bad_transit_]), 'ro', ms = 26, mfc = 'None')
ax2.plot(0,0, 'k*', mfc = "None", ms = 36, mew = 2.)

plt.axhline(0, color = 'k', alpha = 0.3, ls = 'dashed')
plt.axvline(0, color = 'k', alpha = 0.3, ls = 'dashed')
plt.xlabel(r"$\Delta$ Column (pixels)")
#plt.grid()
plt.title('PRF Offset = ' + str("%0.2f" % (1.*(label_))) + ' pixels', fontsize = 18)

#plt.xlim(-0.5, 0.5)
#plt.ylim(-0.5, 0.5)

plt.savefig('/Users/vkostov/Desktop/TIC_' + str(TIC) + '_P' + str(which_period) + '_' + \
            compare_DIFF_to_ + '_Sector' + str(sector_[0]) + '.png')

print(TIC)

In [ ]:
# DO LOMB-SCARGLE ON THE OUT-OF-ECLIPSE

transit_mask_bak = transit_mask_[0].copy()#transit_mask_.flatten().copy()

tt_bak = tt_#[tt_ + tt_ref_ <= 2600.]
ff_bak = ff_#[tt_ + tt_ref_ <= 2600.]
mm_bak = transit_mask_bak#[tt_ + tt_ref_ <= 2600.]

# DO LOMB-SCARGLE!
#
fig = plt.figure(figsize=(20,10))
grid = plt.GridSpec(2, 2, wspace=0.1, hspace=0.3)

mm = ~mm_bak#np.ones(len(tt_bak)).astype(bool)#

plt.subplot(grid[0, :])
plt.plot(tt_bak[~mm] + tt_ref_, ff_bak[~mm], 'r.')
plt.plot(tt_bak[mm] + tt_ref_, ff_bak[mm], 'k.')
#plt.title('TIC ' + TIC + '; Porb = ' + str(np.round(bls_periods[0],4)) + ' days', fontsize = 18)
plt.xlim(min(tt_bak[mm] + tt_ref_), max(tt_bak[mm] + tt_ref_))
#plt.ylim(0.9,1.05)

plt.subplot(grid[1, 0])
from astropy.timeseries import LombScargle
frequency, power = LombScargle(tt_bak[mm], \
                               ff_bak[mm]).autopower(minimum_frequency=1./20., \
                                                  maximum_frequency = 1./1,samples_per_peak=50)
                              # ff_bak[mm]).autopower(minimum_frequency=1./2./bls_periods[0], \
                              #                    maximum_frequency = 1./0.5/bls_periods[0],samples_per_peak=50)

plt.plot(1./frequency, power, 'k')
#plt.xlim(0.5,5)
plt.xlabel('Days', fontsize = 18)
plt.title('Lomb-Scargle Periodogram', fontsize = 18)

plt.subplot(grid[1, 1])
best_frequency = frequency[np.argmax(power)]
t_fit = np.linspace(0, 1)
#ls = LombScargle(tt_[mm], ff_[mm])#, dy)
#y_fit = ls.model(t_fit, best_frequency)
#plt.plot(t_fit, y_fit, 'r')
plt.plot(tt_bak[mm]/(1/best_frequency) % 1, ff_bak[mm], 'k.')
plt.axhline(1., c = 'r', ls = 'dashed', lw = 4)
#plt.plot(tt_[mm]/bls_periods[2] % 1, ff_[mm], 'k.')
plt.title('Prot = ' + str(np.round(1/best_frequency,4)) + ' days', fontsize = 18)
plt.xlim(0.,1)
#plt.ylim(0.9,1.1)

bls_periods[0], 1/best_frequency, 1/best_frequency/bls_periods[0]

In [ ]:
data_TC.tpf_star_y

In [ ]:
data_TC = eleanor.TargetData(star_TC[0], do_psf = False, do_pca = False)#, aperture_mode = 'small')#, height = 11, width = 11)

plt.imshow(np.sum(data_TC.tpf, axis = 0), interpolation='nearest', origin = 'lower', zorder = 0, extent = [0,12,0,12])#, cmap = 'gray')
plt.imshow(data_TC.aperture, cmap = 'pink', alpha = 0.1, zorder = 1, extent = [0,12,0,12])#, cmap = 'brg_r')
plt.plot(data_TC.x_com, data_TC.y_com, 'r.')

In [ ]:
ZZZZ

In [ ]:
# SAVE LIGHTCURVES TO FILE

tt, rr, rr_err, cc, pp, qq = [], [], [], [], [], []

for data in data_TC:
    q=data.quality==0
    tt = np.hstack((tt, data.time[q]))#tt.append(data.time[q])
    rr = np.hstack((rr, data.raw_flux[q]/np.nanmedian(data.raw_flux[q])))
    rr_err = np.hstack((rr_err, data.flux_err[q]))
    cc = np.hstack((cc, data.corr_flux[q]/np.nanmedian(data.corr_flux[q])))
    pp = np.hstack((pp, data.pca_flux[q]/np.nanmedian(data.pca_flux[q])))
    qq = np.hstack((qq, data.psf_flux[q]/np.nanmedian(data.psf_flux[q])))
    
#np.savetxt('TIC_' + str(TIC)+'.csv', np.c_[tt,rr,rr_err,cc,pp,qq], delimiter = ',')
np.savetxt('TIC_' + str(TIC)+'.csv', np.c_[tt,cc,rr_err], delimiter = ',')

In [ ]:
PA, PB, Pout = 2.36, 8.85, 266.5
aA, aB, aout = 8.5, 19.3, 244

MAA, MAB, MBA, MBB = 0.79, 0.74, 0.65, 0.57
Mout, eout, ein = MAA + MAB + MBA + MBB, 0.45, 0.01

binary_ = 'B'

if binary_ == 'A':
    Min = MAA + MAB
    Pin = PA
    ain = aA
elif binary_ ==  'B':
    Min = MBA + MBB
    Pin = PB
    ain = aB

importlib.reload(helper_codes)
sma_ratio, Per_ratio = helper_codes.calculate_stability(Mout, Min, eout, ein)
    
print(Pout/Pin, Per_ratio, Pout/Pin/Per_ratio, aout/ain, sma_ratio, aout/ain/sma_ratio)

In [ ]:
r = 2.4#catalog_['rad'][0]#1.25
dd = 0.005#5e2*1e-6

rx = r*np.sqrt(dd)
print('Rx = ', str(rx), '[RSun]')

In [ ]:
import pickle
fn_ = "Diff_Img_Cube.pkl"
###Load into file
with open(fn_,"wb") as f:
    pickle.dump(diff_cube_,f)
###Extract from file
with open(fn_,"rb") as f:
    diff_cube_saved_ = pickle.load(f)

In [ ]:
#np.savetxt('TIC_' + str(TIC)+'.csv', np.c_[tt_+tt_ref_,ff_], delimiter = ',')
quality_ = qq_ == 0.
np.savetxt('TIC_' + str(TIC)+'.csv', np.c_[tt[quality_], cc_[quality_]], delimiter = ',')

In [ ]:
dep = 10000e-6
delta_mag = 2.5*(np.log10(1 - 2*dep) - np.log10(2*dep))
# delta_mag = 2.5*(np.log10(1 - 2*dep) - np.log10(2*dep))
print(delta_mag)